In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [4]:
bio = pd.read_csv("../dataset/Agriculture_biologique_final.csv", sep ="\t")
env = pd.read_csv("../dataset/Impact environnemental_final.csv", sep="\t",low_memory=False)
qual = pd.read_csv("../dataset/Qualite nutritionnelle_final.csv", sep="\t",low_memory=False)

# 0  Nettoyage

## 0.1 Regroupement des datasets

Colonnes communes env & qual

In [5]:
env.columns[env.columns.isin(qual.columns)]

Index(['code', 'product_name', 'quantity', 'brands_tags', 'countries_tags',
       'serving_size', 'image_url'],
      dtype='object')

Merge des df env et qual

In [6]:
envqual = env.merge(qual, on = ['code', 'product_name', 'quantity', 'brands_tags', 'countries_tags','serving_size', 'image_url'], how='outer', indicator='merged_qual')
envqual.columns

Index(['code', 'product_name', 'quantity', 'brands_tags', 'countries_tags',
       'serving_size', 'image_url', 'ecoscore_grade', 'est_plastique',
       'est_palm', 'est_cocoa', 'ingredients_tags', 'nutriscore_grade',
       'energy-kcal_100g', 'fat_100g', 'saturated-fat_100g', 'sugars_100g',
       'proteins_100g', 'salt_100g', 'nb_nocif', 'merged_qual'],
      dtype='object')

Colonnes communes envqual et bio

In [7]:
envqual.columns[envqual.columns.isin(bio.columns)]

Index(['product_name', 'brands_tags', 'serving_size'], dtype='object')

Merge des df env et qual

In [8]:
envqualbio = envqual.merge(bio, on=['product_name', 'brands_tags', 'serving_size'], how='outer', indicator='merged_bio')
envqualbio.columns

Index(['code', 'product_name', 'quantity', 'brands_tags', 'countries_tags',
       'serving_size', 'image_url', 'ecoscore_grade', 'est_plastique',
       'est_palm', 'est_cocoa', 'ingredients_tags', 'nutriscore_grade',
       'energy-kcal_100g', 'fat_100g', 'saturated-fat_100g', 'sugars_100g',
       'proteins_100g', 'salt_100g', 'nb_nocif', 'merged_qual', 'est_bio',
       'merged_bio'],
      dtype='object')

On supprime les doublons

In [9]:
envqualbio = envqualbio.drop_duplicates()
envqualbio.duplicated().sum()

0

In [10]:
envqualbio.reset_index(inplace=True)

In [11]:
envqualbio.drop(columns='index',inplace=True)

Gestion des valeurs manquantes

Pourcentage des valeurs manquantes par colonnes

In [12]:
envqualbio.isnull().sum()

code                       0
product_name           24354
quantity              579572
brands_tags           412045
countries_tags             0
serving_size          763369
image_url              71750
ecoscore_grade           662
est_plastique              0
est_palm                   0
est_cocoa                  0
ingredients_tags      596585
nutriscore_grade      543571
energy-kcal_100g      215440
fat_100g              197506
saturated-fat_100g    192288
sugars_100g           192913
proteins_100g         195686
salt_100g             222287
nb_nocif                   0
merged_qual                0
est_bio               412045
merged_bio                 0
dtype: int64

In [13]:
envqualbio.shape

(849616, 23)

In [14]:
envqualbio.isnull().sum() / envqualbio.shape[0] * 100

code                   0.000000
product_name           2.866471
quantity              68.215759
brands_tags           48.497792
countries_tags         0.000000
serving_size          89.848708
image_url              8.444992
ecoscore_grade         0.077918
est_plastique          0.000000
est_palm               0.000000
est_cocoa              0.000000
ingredients_tags      70.218193
nutriscore_grade      63.978433
energy-kcal_100g      25.357338
fat_100g              23.246502
saturated-fat_100g    22.632342
sugars_100g           22.705905
proteins_100g         23.032288
salt_100g             26.163231
nb_nocif               0.000000
merged_qual            0.000000
est_bio               48.497792
merged_bio             0.000000
dtype: float64

# Quelles sont les marques qui jouent le jeu et affichent le nutriscore ? Lesquelles ne le font pas?

On enleve les marques qui ne sont pas répertoriées

In [16]:
ind_brand_missing = envqualbio.loc[envqualbio.brands_tags.isnull()].index
df_sansmarques= envqualbio.drop(index=ind_brand_missing)

Selection des marques ayant le plus de produits sans nutriscores

In [18]:
bad_marques = df_sansmarques[['brands_tags','nutriscore_grade']][df_sansmarques.nutriscore_grade.isnull()]
bad_marques.brands_tags.value_counts()

brands_tags
carrefour                           5935
auchan                              3911
u                                   2035
nestle                              1687
casino                              1554
                                    ... 
crystal-gourmet                        1
la-maziere                             1
moulin-artisanal-gilles-matignon       1
mon-rhum                               1
vitaflor,milical                       1
Name: count, Length: 36779, dtype: int64

On peut se poser la question de la pertinence de ce résultat, pour une marque qui ne dispose que d'un produit ( ou très peu ) ce sera souvent des marques locales, qu'un consommateur ne va pas forcémenet retrouver chez lui en grande surface. Et donc si ce produit n'a pas de nutriscore, la marque aura tout de suite un % de son produit sans nutriscore très élevé.

In [25]:
df_sansmarques.brands_tags.value_counts()

brands_tags
carrefour                                                                                    12331
auchan                                                                                       10813
u                                                                                             6520
casino                                                                                        4809
leader-price                                                                                  4473
                                                                                             ...  
carrefour,carrefour-selection,cmi-carrefour-marchandises-internationales,groupe-carrefour        1
carrerfour                                                                                       1
carrefour,persyn                                                                                 1
carrefour-discount,carrefour-discount,carrefour                                                  

In [27]:
a = df_sansmarques.brands_tags.map(lambda x: 'carrefour' in x and).value_counts()
a

brands_tags
False    421618
True      15953
Name: count, dtype: int64

# Quelles sont les marques qui ont le plus recours aux additifs nocifs ?

In [88]:
envqualbio.nb_nocif.unique()

array([0., 1., 2.])

In [87]:
envqualbio.groupby('brands_tags').nb_nocif.sum().sort_values(ascending= False)[:10]

brands_tags
sodastream          29.0
nestle,sveltesse    10.0
a-rom                9.0
dove                 8.0
nestle               7.0
schweppes            7.0
freeway              6.0
eco                  4.0
fruiss               4.0
steff                4.0
Name: nb_nocif, dtype: float64

In [ ]:
test = envqualbio.copy()
def modif_nut(row):
    if row.nutriscore_grade is in ['a','d', 'e', 'b', 'c'] : 
        return row
    elif 
test.apply(modif_nut)

In [109]:
envqualbio.nutriscore_grade.unique()

array(['a', nan, 'd', 'e', 'b', 'c'], dtype=object)

In [105]:
envqualbio.loc[envqualbio[['brands_tags','product_name']].duplicated()]

AttributeError: 'DataFrame' object has no attribute 'unique'